In [ ]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq

import os
import matplotlib.pyplot as plt
import json
from pathlib import Path
import pickle

# from coffea.nanoevents import NanoEventsFactory, BaseSchema
# import uproot
# import awkward as ak
# import hist
# from plotting import * 
from tqdm import tqdm

# from agc_schema import AGCSchema

import mplhep as hep
plt.style.use([hep.style.CMS])

In [ ]:
pmap = {}

pmap['QCD'] = [
    #'QCD_HT-100to200', 'QCD_HT-200to400', 'QCD_HT-400to600', 
    'QCD_HT-600to800', 'QCD_HT-800to1000', 'QCD_HT-1000to1200', 
    'QCD_HT-1200to1500', 'QCD_HT-1500to2000', #'QCD_HT-2000'
    ]

# pmap['Zjets_had_1j'] = [
#     'Zto2Q-2Jets_PTQQ-100to200_1J', 'Zto2Q-2Jets_PTQQ-200to400_1J',
#     'Zto2Q-2Jets_PTQQ-400to600_1J', 'Zto2Q-2Jets_PTQQ-600_1J'
#     ]

# pmap['Zjets_had_2j'] = [
#     'Zto2Q-2Jets_PTQQ-100to200_2J', 'Zto2Q-2Jets_PTQQ-200to400_2J',
#     'Zto2Q-2Jets_PTQQ-400to600_2J', 'Zto2Q-2Jets_PTQQ-600_2J'
#     ]

# pmap['Wjets_had_1j'] = [
#     'Zto2Q-2Jets_PTQQ-100to200_1J', 'Zto2Q-2Jets_PTQQ-200to400_1J',
#     'Zto2Q-2Jets_PTQQ-400to600_1J', 'Zto2Q-2Jets_PTQQ-600_1J'
#     ]

# pmap['Wjets_had_2j'] = [
#     'Wto2Q-2Jets_PTQQ-100to200_2J', 'Wto2Q-2Jets_PTQQ-200to400_2J',
#     'Wto2Q-2Jets_PTQQ-400to600_2J', 'Wto2Q-2Jets_PTQQ-600_2J'
#     ]

pmap['Wto2Q'] = ['Wto2Q-3Jets_HT-200to400', 'Wto2Q-3Jets_HT-400to600',
                 'Wto2Q-3Jets_HT-600to800','Wto2Q-3Jets_HT-800'
                ]

pmap['Zto2Q'] = ['Zto2Q-4Jets_HT-200to400', 'Zto2Q-4Jets_HT-400to600',
                 'Zto2Q-4Jets_HT-600to800','Zto2Q-4Jets_HT-800'
                ]

pmap['VBF_Vto2Q'] = ['VBFWto2Q','VBFWtoLNu','VBFZto2L','VBFZto2Nu','VBFZto2Q']

pmap['VV'] = ['WW', 'WZ', 'ZZ']

pmap['ggF'] = ['GluGluHto2B_PT-200_M-125']

pmap['VBF'] = ['VBFHto2B_M-125_dipoleRecoilOn']

pmap['VH'] = ['WminusH_Hto2B_Wto2Q_M-125', 'WplusH_Hto2B_Wto2Q_M-125'] +['ZH_Hto2B_Zto2Q_M-125','ggZH_Hto2B_Zto2Q_M-125']

pmap['WH'] = ['WminusH_Hto2B_Wto2Q_M-125', 'WplusH_Hto2B_Wto2Q_M-125',
              #'WminusH_Hto2B_WtoLNu_M-125', 'WplusH_Hto2B_WtoLNu_M-125'
             ]

pmap['ZH'] = ['ZH_Hto2B_Zto2Q_M-125', #'ZH_Hto2B_Zto2L_M-125', 'ZH_Hto2B_Zto2Nu_M-125',
              'ggZH_Hto2B_Zto2Q_M-125', #'ggZH_Hto2B_Zto2L_M-125', #'ggZH_Hto2B_Zto2Nu_M-125'
             ]

pmap['ttH'] = ['ttHto2B_M-125']

pmap['TopFS'] = ['TTto4Q', 'TTto2L2Nu', 'TTtoLNu2Q']+['TWminusto2L2Nu','TWminusto4Q','TWminustoLNu2Q']+ ['TbarWplusto2L2Nu', 'TbarWplusto4Q','TbarWplustoLNu2Q']+ ['TbarBQ_t-channel_4FS', 'TBbarQ_t-channel_4FS']


pmap['ttbar'] = ['TTto4Q', 'TTto2L2Nu', 'TTtoLNu2Q']

pmap['T'] = ['TWminusto2L2Nu','TWminusto4Q','TWminustoLNu2Q']

pmap['Tbar'] = ['TbarWplusto2L2Nu', 'TbarWplusto4Q','TbarWplustoLNu2Q']

pmap['TB'] = ['TbarBQ_t-channel_4FS', 'TBbarQ_t-channel_4FS']


pmap['Jet_data'] = ['JetMET_Run2023Cv1', 'JetMET_Run2023Cv2', 
                    'JetMET_Run2023Cv3', 'JetMET_Run2023Cv4',]

pmap['Muon_data'] = ['Muon_Run2023Cv1', 'Muon_Run2023Cv2', 
                    'Muon_Run2023Cv3', 'Muon_Run2023Cv4',]
# pmap['TopFS'] = pmap['ttbar']+pmap['T']+ pmap['Tbar']+ pmap['TB']
pmap['Hbb_all'] = pmap['VBF'] + pmap['VH'] + pmap['ggF'] + pmap['ttH']
    # 'ggZH_Hto2B_Zto2L_M-125', 'ggZH_Hto2B_Zto2Nu_M-125', 'ggZH_Hto2B_Zto2Q_M-125',
    # 'GluGluHto2B_M-125', 'ttHto2B_M-125', 'VBFHto2B_M-125',
    # 'WminusH_Hto2B_Wto2Q_M-125', 'WminusH_Hto2B_WtoLNu_M-125', 
    # 'WplusH_Hto2B_Wto2Q_M-125', 'WplusH_Hto2B_WtoLNu_M-125',
    # 'ZH_Hto2B_Zto2L_M-125', 'ZH_Hto2B_Zto2Nu_M-125', 'ZH_Hto2B_Zto2Q_M-125'
    

pmap['background'] = pmap['Wto2Q']+pmap['Zto2Q'] +pmap['VV']+pmap['TopFS']+pmap['QCD']

print(pmap['VH'])

pmap_file = './samples.json'

with open(pmap_file, 'w') as json_file:
    json.dump(pmap, json_file)


In [ ]:
'''Cut definitions'''
def minjetkin(df):
    # fatjets = df['ak8FatJetmsoftdrop', 'ak8FatJetPt', 'ak8FatJetEta']
    # print(df['ak8FatJetmsoftdrop'].shape)
    fatjet_msd = df['FatJet0_msd'].values
    fatjet_pt = df['FatJet0_pt'].values
    fatjet_eta = df['FatJet0_eta'].values
        # fatjets['msdcorr'] = fatjets.msoftdrop
        # fatjets['qcdrho'] = 2 * np.log(fatjets.msdcorr / fatjets.pt)
    candidatejet = df[
            (fatjet_pt > 200)
            & (abs(fatjet_eta) < 2.5)
            # & fatjets.isTight 
        ]

    # candidatejet = candidatejet[:, :2]
    # candidatejet = ak.firsts(candidatejet[ak.argmax(candidatejet.particleNet_XbbVsQCD, axis=1, keepdims=True)])

    # bvl = candidatejet.particleNet_XbbVsQCD
    minjetkin=np.array([
            (candidatejet['FatJet0_pt'] >= 450)
            & (candidatejet['FatJet0_pt']< 1200)
            & (candidatejet['FatJet0_msd'] >= 40.)
            & (candidatejet['FatJet0_msd'] < 201.)
            & (abs(candidatejet['FatJet0_eta']) < 2.5)
       ])
    # minjetkin=np.sum(minjetkin, axis=1).astype('bool').transpose()
    minjetkin = minjetkin.astype('bool').transpose()

    # print(minjetkin)
    # print(minjetkin.shape)
    # print(minjetkin)
    
    return df[minjetkin]

In [ ]:
'''Data Aquisition'''
def get_paths(year, data_path, proc = 'QCD', deep=False):
    #returns list of paths to parquet files
    #DASK VERSION: parquet_parents = [os.path.join(data_path, year, f'{year}_{p}', 'signal-all') for p in pmap[proc]]
    parquet_parents = [os.path.join(data_path, year, p, 'parquet','signal-all') for p in pmap[proc]]

    if deep:
        file_list=None
        for parent in parquet_parents:
            if file_list is None:
                file_list = [os.path.join(parent,file)for file in os.listdir(parent)]
            else:
                file_list = np.append(file_list, [os.path.join(parent,file)for file in os.listdir(parent)])
    else:
        file_list=parquet_parents
    return file_list

def get_proc_field(file_path, field = 'GenVMass', cut=True):
    events = pd.read_parquet(file_path)
    if cut:
        # print('before: ', events.shape)
        # print(events)
        events = minjetkin(events)
        # print('after: ', events.shape)
    
    if field in events.columns:
       variable = events[field].values
    else:
        print(f"Field {field} not found")

    del events
    # print('var', variable.shape)

    return variable

def get_sum_genweights(data_dir: Path, dataset: str) -> float:
    """
    Get the sum of genweights for a given dataset.
    :param data_dir: The directory where the datasets are stored.
    :param dataset: The name of the dataset to get the genweights for.
    :return: The sum of genweights for the dataset.
    """
    total_sumw = 0
    try:
        # Load the genweights from the pickle file
        for pickle_file in list(Path(data_dir / dataset / "pickles").glob("*.pkl")):
            with Path(pickle_file).open("rb") as file:
                out_dict = pickle.load(file)
            # The sum of weights is stored in the "sumw" key
            # You can access it like this:
            for key in out_dict:
                sumw = next(iter(out_dict[key]["sumw"].values()))
            total_sumw += sumw
    except:
        warnings.warn(
            f"Error loading genweights for dataset: {dataset}. Skipping.",
            category=UserWarning,
            stacklevel=2,
        )
        total_sumw = 1

    # print(f"Total sum of weights for all pickles for {dataset}: {total_sumw}")
    return total_sumw
    

def fill_hist(year, process, field, cut = True, full=True):
    
    data_dir = '/uscms/home/bweiss/nobackup/hbb/' #folder containing all samples
    #collect all field data for the process
    dirs = get_paths(year, data_dir, process)
    print(dirs)
    # print(dirs)
    var = None

    for d in tqdm(dirs, desc="Processing "+str(process)+' '+str(field)):
        # print(d)
        for i, file in enumerate(os.listdir(d)):
            path = os.path.join(d,file)
            # print(path)
            # print(path)
            if var is None:
                var = get_proc_field(path, field, cut=cut)
                # print(var)
            else:
                var = np.append(var, get_proc_field(path, field, cut=cut), axis=0)
                # print(var.shape, type(var))
            # if not full:
            #     print('stored var from only 1 parquet')
            #     break
    # var = var.flatten()
    return var

def get_keys_from_value(d, val):
    return [k for k in d.keys() if val in d[k]]

def path_to_parquet(data_dir, year, MC_name):
    return os.path.join(data_dir, year, MC_name, 'parquet', 'signal-all')

def get_var(path = './', MC_name = '', cols = [''], preselect = True):
    '''get a particular collumn of an MC sample or parquet together with its true weight'''
    if os.path.isfile(path):
        paths = [path]
    else:
        paths = os.listdir(path)
    var = pd.DataFrame()
    for p in paths:
        preselect_cols =['FatJet0_msd', 'FatJet0_pt', 'FatJet0_eta', 'weight']
        preselect_cols += cols
        preselect_cols = list(set(preselect_cols))
        # if col not in preselect_cols:
        #     preselect_cols += cols
        sumW_file = './sumW.json'
        with open(sumW_file, 'r') as json_file:
            sumW_dict = json.load(json_file)
            sumW = sumW_dict[MC_name]
        thisdf = pd.read_parquet(os.path.join(path,p), columns = preselect_cols)
        # N = len(thisdf)
        thisdf = minjetkin(thisdf)
        thisdf['weight_final'] = thisdf['weight']/sumW
        # print(get_keys_from_value(pmap, MC_name))
        thisdf['MC_cat'] = get_keys_from_value(pmap, MC_name)[0]
        var = pd.concat([var, thisdf[['weight_final', 'MC_cat']+cols]], axis = 0, ignore_index=True)
    return var

# print(get_dirs(data_dir, proc, field = 'Zjets_had_1j'))

In [ ]:
toPlot = pmap['Hbb_all'] + pmap['background']
# print(toPlot)
var = 'FatJet0_msd'
data_path = '/uscms/home/bweiss/nobackup/hbb/'
year = '2023'

MC_cats = ['VBF', 'VH', 'ggF', 'ttH']
MC_cats += ['QCD','Wto2Q','Zto2Q','VV','TopFS']
# MC_cats = ['TopFS']
colors = ['green', 'orange', 'red', 'blue', 'cyan', 'pink', 'grey', 'purple', 'brown']

jetN = 1
# cols = [f'FatJet{jetN}_pnetTXbb', f'FatJet{jetN}_pnetTXcc',
#        f'FatJet{jetN}_pnetTXqq', f'FatJet1_pnetTXgg', ]
cols = ['FatJet0_n2b1', 'FatJet0_n3b1', 'FatJet0_pnetTXcc', 'FatJet0_msdmatched',]

hist = pd.DataFrame()
for sample in tqdm(toPlot): 
    MC_cat = get_keys_from_value(pmap, sample)[0]
    print(f'Fetching {var} from {sample} in {MC_cat}', end="")
    parquets_path = path_to_parquet(data_path, year, sample)
    v = get_var(path = parquets_path, MC_name = sample, cols = cols)
    hist = pd.concat([hist,v], axis = 0, ignore_index=True)
print(hist)

In [ ]:
all_bins = [
        # np.linspace(25,205, 18), np.linspace(25,205, 18), #msd
        # np.linspace(200,1000, 16), np.linspace(200,1000, 16), #pt
        # np.linspace(-3,3, 18), np.linspace(-3,3, 18), #eta
        # np.linspace(-3.2,3.2, 18), np.linspace(-3.2,3.2, 18), #phi
        np.linspace(0,0.5, 20), np.linspace(0,5, 18), 
        np.linspace(0,1, 20), np.linspace(25,205, 18)
       ]

fig, ax = plt.subplots(2,2, figsize=(12,10))
ax = ax.flat
# cols = ['FatJet0_eta', 'FatJet1_eta', 'FatJet0_phi','FatJet1_phi']

for j, var in tqdm(enumerate(cols)):
    bins = all_bins[j]
    # bins = np.linspace(0,15,16) #nJet
    # print(bins)
    # print(max(hist['FatJet0_msd']))
    for i, cat in enumerate(MC_cats):
        cat_mask = hist['MC_cat'] == cat
        ax[j].hist(hist[cat_mask][var], bins = bins, lw=2, histtype = 'step', label = cat, weights = abs(hist[cat_mask]['weight_final']), color = colors[i])
    ax[j].set(xlabel=var)
    ax[j].set(yscale='log')
    
fig.legend(MC_cats,loc = 'lower center', ncol=5, bbox_to_anchor=(0.5, -0.1 ))
fig.supylabel('Weighted events/bin')

fig.tight_layout()

save_dir = './InputVar_plots'
fig.savefig(os.path.join(save_dir,'FatJet0_nb_cc_msdmatched.png'))
# plt.yscale('log')
# # plt.xlim([0,210])

# plt.ylabel('yield')

In [ ]:
print(len(hist))
MC_cats = ['VBF', 'VH', 'ggF', 'ttH']
MC_cats += ['QCD','Wto2Q','Zto2Q','VV','TopFS']
# MC_cats = ['TopFS']
colors = ['green', 'orange', 'red', 'blue', 'cyan', 'pink', 'grey', 'purple', 'brown']

cols = ['FatJet0_msd', 'FatJet1_msd', 'FatJet0_pt','FatJet1_pt']

fig = plt.subplots((figsize=(12,9))
bins = np.linspace(25,205, 18)
# bins = np.linspace(0,15,16) #nJet
# print(bins)
# print(max(hist['FatJet0_msd']))
for i, cat in enumerate(MC_cats):
    cat_mask = hist['MC_cat'] == cat
    plt.hist(hist[cat_mask][var], bins = bins, lw=2, histtype = 'step', label = cat, weights = abs(hist[cat_mask]['weight_final']), color = colors[i])
plt.legend()
# plt.yscale('log')
plt.xlim([0,210])
plt.xlabel(var)
plt.ylabel('yield')

# print(v)

In [ ]:
'''Get sumW for each MC sample and dump it into a json file'''
sample_dir = '/uscms/home/bweiss/nobackup/hbb/2023'
sumW = {}
for file in os.listdir(sample_dir):
    if ('JetMET' not in file) & ('Muon' not in file):
        sample_sumW = get_sum_genweights(Path(sample_dir), file)
        sumW[file] = sample_sumW
        print(file, sample_sumW)
sumW_file = './sumW.json'
with open(sumW_file, 'w') as json_file:
    json.dump(sumW, json_file)

In [ ]:
# dirs = get_paths('2022', '/uscms/home/bweiss/nobackup/hbb/', 'ggF')
var = fill_hist('2023', 'VH', 'Jet1_mass', cut=True)

print(type(var))
# plt.hist(var)

In [ ]:
'''print fields for a given process'''

proc = 'VBF'
main_dir = '/uscms/home/bweiss/nobackup/hbb/'
g = get_paths('2022'main_dir, proc)
df = pd.read_parquet(g[0])
print(df.columns)

# fields = np.unique(np.array([c[0] for c in df.columns]))
# sizes = np.array([[c[0] for c in df.columns].count(f) for f in fields])

# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)


# print(pd.DataFrame({'Field':fields , 'Size':sizes}))


In [ ]:
procs = [
        'ggF', 'ggF'
        # 'QCD'
        ]

fields = [
        # 'GenbPt', 'GenbEta', 'GenbPhi'
        'FatJet1_pt', #'FatJet1_phi', #'FatJet1_eta', 'FatJet1_msd'
        'Jet0_pt', 'Jet1_pt','Jet2_pt'
    ]
cut = [False, True]
# scale= ['log','log','log','log']
colors = ['red', 'blue', 
         ]
# var = fill_hist(proc,field)[:,:2]

fig, ax = plt.subplots(2,2, figsize = (8,8))
plt.tight_layout(pad=2)

ax = ax.flat

for i, field in enumerate(fields):
    for j, proc in enumerate(procs):
        # print(field)
        # field = fields[i]
        var = fill_hist('2022', proc, field, cut = cut[j])
        # print(proc, field, f'cut={cut[j]}', var.shape)
        print(len(var), var)
        # non_nan = ~np.isnan(var)
        # print(var[non_nan], len(var[non_nan]))
        # if i == 0:
        # print(field, var[:5], var[high_pt][:5]) 
        
        # if 'pt' in field:
        #     bins = np.linspace(200,1000, 18)
        #     # bins = []
        #     ax[i].hist(var, histtype='step', bins=bins, label = proc, color = colors[j], linewidth=2)
        #     ax[i].set(yscale = 'log', xlabel = field)
        # elif 'msd' in field:
        #     bins = np.linspace(0,300, 10)
        #     ax[i].hist(var, histtype='step', bins=bins, label = proc, color = colors[j], linewidth=2)
        #     ax[i].set(xlabel = field)
        # else:
        #     bins = np.linspace(0,1,10)
        #     ax[i].hist(var, histtype='step', bins=bins, label = proc, color = colors[j], linewidth=2)
        #     ax[i].set(xlabel = field)
            # if scale[i]:
            #     ax[i].set(yscale=scale[i])
            
        # ax[1][i].hist(var[low_pt], histtype='step', density=True)
        # ax[1][i].set(xlabel = 'Sub-leading '+field)
    # del pt, var
nl = len(procs)
lines_labels = [a.get_legend_handles_labels() for a in fig.axes]
lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
lgd = fig.legend(lines[:nl], labels[:nl], 
                 # bbox_to_anchor=(1.05, 0.5),
                 loc='upper right',
                 ncol=3
                )
fig.supylabel('Events')
fig.suptitle('Input Variable Scan Checking Cuts', ha='left', x=0.01)

plt.tight_layout()


In [ ]:
procs = [
        #'ggF', 'VBF', 
        'VH', 
        # 'QCD'
        ]

fields = [
        # 'GenbPt', 'GenbEta', 'GenbPhi'
        'Jet0_btagPNetB', 'Jet0_btagPNetQvG','Jet0_btagPNetCvL', 'Jet0_btagPNetCvB' 
    ]
scale= ['log','log','log','log']
colors = ['red', 'blue', 'green', 'black']
# var = fill_hist(proc,field)[:,:2]

fig, ax = plt.subplots(2,2, figsize = (8,8))
plt.tight_layout(pad=2)

ax = ax.flat

# pt = ak.Array(fill_hist(proc,field))
for j, proc in enumerate(procs):
    # pt = fill_hist(proc,'GenbPt')[:,:2]
    # pt = fill_hist(proc,'ak8FatJetPt')[:,:2]
    # high_pt = (np.arange(len(pt[:,0])), np.argmax(pt, axis = 1))
    # low_pt = (np.arange(len(pt[:,0])), np.argmin(pt, axis = 1))
    # mask = np.zeros(pt.shape) 
    # mask[high_pt] = True
    # print(mask)
    # print(high_pt)
    # print(proc)
    for i, field in enumerate(fields):
        # print(field)
        # field = fields[i]
        var = fill_hist('2022', proc, field, cut=True)
        print(len(var))
        # var = list(var)
        # print(var[:10])
        # if i == 0:
        # print(field, var[:5], var[high_pt][:5]) 
        if 'Pt' in field:
            bins = np.linspace(200,1000, 10)
            ax[i].hist(var, histtype='step', density=True, bins=bins, label = proc, #color = colors[j], 
                       linewidth=2)
            ax[i].set(yscale = 'log', xlabel = field)
        elif 'soft' in field:
            bins = np.linspace(0,300, 10)
            ax[i].hist(var, histtype='step', density=True, bins=bins, label = proc, #color = colors[j], 
                       linewidth=2)
            ax[i].set(xlabel = field)
        else:
            bins = np.linspace(0,1,10)
            ax[i].hist(var, histtype='step', density=True, bins=bins, label = proc, #color = colors[j], 
                       linewidth=2)
            ax[i].set(xlabel = field)
            if scale[i]:
                ax[i].set(yscale=scale[i])
            
        # ax[1][i].hist(var[low_pt], histtype='step', density=True)
        # ax[1][i].set(xlabel = 'Sub-leading '+field)
    # del pt, var
nl = len(procs)
lines_labels = [a.get_legend_handles_labels() for a in fig.axes]
lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
lgd = fig.legend(lines[:nl], labels[:nl], 
                 # bbox_to_anchor=(1.05, 0.5),
                 loc='upper right',
                 ncol=3
                )
fig.supylabel('Events (Normalized)')
fig.suptitle('Leading Ak4 Jet flavor', ha='left', x=0.01)

plt.tight_layout()
